In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
import scipy
import csv

#import deeplabcut


Autosaving every 180 seconds


In [3]:
video_name = "/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi"

data= np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/traces_means.npz')

tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']
print ("DONE LOADING")

DONE LOADING


In [4]:
ids = np.arange(4)
for id_ in ids:
    for k in range(tracesx.shape[0]):
        if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
            print (k, tracesx[k,id_], tracesy[k,id_])
            tracesx[k,id_]= tracesy[k,id_] =np.nan
            print (k, tracesx[k,id_], tracesy[k,id_])
            print ('')

3135 1123.123 nan
3135 nan nan

14715 nan 92.174
14715 nan nan

779 nan 625.86
779 nan nan

13534 288.423 nan
13534 nan nan

13963 nan 244.162
13963 nan nan

9318 317.532 nan
9318 nan nan

9789 305.184 nan
9789 nan nan

14057 312.99476045543315 nan
14057 nan nan



In [5]:
# BREAKUP LARGE JUMPS
tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']

def delete_jumps(tracesx, tracesy):
    #tracesx_ip=tracesx.copy()
    #tracesy_ip=tracesy.copy()
    for k in range(tracesx.shape[1]):

        # for big jumps, set intermediate values to nans
        max_jump = 50
        for p in range(0,tracesx.shape[0]-1,1):
            distx = tracesx[p,k]-tracesx[p+1,k]
            disty = tracesy[p,k]-tracesy[p+1,k]
            dist = np.sqrt(distx**2+disty**2)
            #print ("Dist; ", dist)
            if dist>max_jump:
                #print (snippets[p,0],snippets[p,1])
                tracesx[p:p+2,k]= np.nan
                tracesy[p:p+2,k]= np.nan
                #print ("found large jump: ", p/25.)
            #else:
            #    print (dist, "not merging")


       # break
        #np.isnan(x_loc) or np.isnan(y_loc):
    print ("DONE FIXING JUMPS")
    #print ("tracex: ", tracesx[:100,0])
    
    return tracesx, tracesy

tracesx, tracesy = delete_jumps(tracesx,tracesy)
#print ("tracex_ip: ", tracesx_ip[:100,0])

#tracesx = tracesx_ip.copy()
#tracesy = tracesy_ip.copy()


DONE FIXING JUMPS


In [6]:
# INTERPOLATE TRACES
# tracesx = data['tracesx_mean']
# tracesy = data['tracesy_mean']

# TODO Add time threshold also; 
def interpolate_traces(tracesx, tracesy, dist_threshold=10., time_threshold=10):
    print (tracesx.shape)
    tracesx_ip=tracesx.copy()
    tracesy_ip=tracesy.copy()
    for k in range(tracesx_ip.shape[1]):
        start = False
        end = False
        idx = np.where(~np.isnan(tracesx_ip[:,k]))[0]

        # make snippets
        snippets=[]
        locs = []
        start = idx[0]
        #for p in range(1,idx.shape[0],1):
        p=1
        ctr=0
        while True:
            if idx[p]-idx[p-1]>1:
                end = idx[p-1]
                snippets.append([start,end])
                start = idx[p]   

                # find locations of start and end
                locs.append([])
                locs[ctr].append([tracesx_ip[start,k], tracesy_ip[start,k]])
                locs[ctr].append([tracesx_ip[end,k], tracesy_ip[end,k]])

                # bump 2 over
                p+=2
                ctr+=1
            else:
                p+=1

            if p>=idx.shape[0]:
                break

        snippets = np.array(snippets)
        #print (snippets.shape)
        #print (snippets)
        locs = np.array(locs)
    #     print (locs.shape)
    #     print (locs[0])

        # fill in space between snippets if euclidean distance < threshold
        #threshold = 10.
        # AND IF NOT TOO FAR APART; 
        for p in range(snippets.shape[0]-1):
            # Skip if interpolate across too long a sgement
            if (snippets[p,0] - snippets[p-1,1])>time_threshold:
                continue
            dist = np.linalg.norm(locs[p,1]-locs[p+1,0])
            
            print ("Dist; ", dist, locs[p,1],locs[p+1,0])
            if dist<dist_threshold:
                #print (snippets[p,0],snippets[p,1])
                tracesx_ip[snippets[p-1,1]:snippets[p,0]+1,k]= tracesx_ip[snippets[p-1,1],k]
                tracesy_ip[snippets[p-1,1]:snippets[p,0]+1,k]= tracesy_ip[snippets[p-1,1],k]
            #else:
            #    print (dist, "not merging")

    return tracesx_ip, tracesy_ip

dist_threshold = 10.
time_threshold = 10
tracesx, tracesy = interpolate_traces(tracesx, tracesy, dist_threshold, time_threshold)

(16572, 4)
Dist;  77.49147358419505 [325.412  637.1085] [316.006 714.027]
Dist;  13.659631010172069 [322.616      716.21919249] [323.381 702.581]
Dist;  17.409310842190077 [324.429 703.659] [308.719 711.161]
Dist;  10.437616212449713 [307.09936231 708.272     ] [316.659      712.46212642]
Dist;  0.6504101626249881 [316.71971018 712.69034677] [316.68317072 712.0409638 ]
Dist;  0.3281655925403125 [316.56917072 712.31112642] [316.883      712.40706307]
Dist;  8.541942184650976 [309.373 710.154] [317.51      712.7528473]
Dist;  5.027379888353558 [309.197 710.327] [311.96158813 714.526     ]
Dist;  6.638673165059635 [318.20875139 712.906     ] [324.573 714.795]
Dist;  4.191504626206979 [320.06862094 714.32493602] [324.25948872 714.398     ]
Dist;  5.423648554843573 [324.51  714.408] [319.12347594 713.7745    ]
Dist;  0.6296107652986601 [324.58757604 714.389     ] [325.15  714.672]
Dist;  0.17819558089928964 [319.85732795 713.92057599] [319.91254725 714.09      ]
Dist;  6.346739050896269 [31

Dist;  11.772620778739093 [273.365 237.645] [283.715 232.035]
Dist;  5.95487875611252 [267.874 237.21 ] [268.115 231.26 ]
Dist;  15.104136645845685 [283.828 231.847] [269.60056514 236.918     ]
Dist;  5.301281095169378 [268.185 231.096] [271.2925 235.391 ]
Dist;  2.5346615512887145 [275.54128337 237.00914087] [274.61363941 239.36795113]
Dist;  10.995559217247676 [271.4995 231.2265] [275.238 241.567]
Dist;  8.360649346667019 [275.405 241.696] [276.66869496 233.43140488]
Dist;  0.764885612363082 [274.631 241.484] [275.286 241.089]
Dist;  4.108711263888004 [274.617  238.5205] [276.683 242.072]
Dist;  1.594067125311854 [275.548 241.422] [277.137 241.549]
Dist;  2.8715713468413235 [276.97  242.798] [274.139 242.317]
Dist;  7.488166756963292 [274.48147684 241.901     ] [277.463 235.032]
Dist;  4.53105695364593 [273.18170243 242.9745    ] [271.9    238.6285]
Dist;  2.7069777148908076 [278.406      234.75926719] [275.719 234.431]
Dist;  8.9575190165101 [275.013 241.892] [275.22135458 232.93690

Dist;  47.63891963772897 [565.8535  88.132 ] [569.4955 135.6315]
Dist;  143.7450353264417 [758.171 637.766] [873.841 723.107]
Dist;  826.3863733296912 [574.096 132.675] [1152.205  723.187]
Dist;  290.5607714847611 [898.001  720.2145] [1150.2904033  576.077    ]
Dist;  93.17746681735885 [1138.857  566.985] [1054.6905  527.0095]
Dist;  532.1494062713862 [1130.96487347  561.556     ] [601.56198098 507.56      ]
Dist;  755.0339923877601 [1055.628  536.301] [315.922 687.666]
Dist;  167.47499310976391 [458.8145 612.2245] [314.63678179 697.434     ]
Dist;  679.3262208845247 [313.254 712.984] [968.4585 533.565 ]
Dist;  795.3035650280025 [315.2455 700.663 ] [1059.016  419.037]
Dist;  22.051408554783986 [1063.7035  438.335 ] [1059.014  459.882]
Dist;  17.02331601069539 [1058.366  418.155] [1055.284  401.413]
Dist;  122.52708816012884 [1060.494  435.139] [1001.807  542.697]
Dist;  278.23143268462377 [1106.265  442.745] [953.28774506 675.147     ]
Dist;  50.51987498180393 [921.14330568 659.524    

In [7]:
# AUTO SWITCH LOCATIONS BASED ON PROXIMTIY



In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

global current_index, selected_animal, tracesx, tracesy

# reload data
#tracesx = tracesx_ip.copy() #data['tracesx_mean'].copy()
#tracesy = tracesy_ip.copy() #data['tracesy_mean'].copy()


# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250
history = np.ones((4,5),'int32')+100

# save current frame of video
current_index = []
current_index.append(0)

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)

# initialize start and ends of video to be analyzed
frame_rate = 25
original_vid = cv2.VideoCapture(video_name)

# initialize video
loc = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
ret, frame = original_vid.read()
n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize scatter plots
ax_video = plt.subplot(gs[:, 1:4])
img = ax_video.imshow(frame)
ax_video.set_xlim(150,1200)
ax_video.set_ylim(50,800)

axes_animals=[]
for k in range(4):
    x_loc = tracesx[0,k]
    y_loc = tracesy[0,k]
    if np.isnan(x_loc) or np.isnan(y_loc):
        x_loc=100
        y_loc=100
    axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

# plot traces
ax_tracesx = plt.subplot(gs[:2, :1])
ax_tracesx.set_title("DLC-x")
init_length = 10*25
t = np.arange(init_length)/25.
ax1 = []
for k in range(4):
    temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
    ax1.append(temp)

ax_tracesx.set_xticks([])
ax_tracesx.set_ylim(0,1280)

# plot y traces
ax_tracesy = plt.subplot(gs[2:, :1])
ax_tracesy.set_title("DLC-y")
ax2 = []
for k in range(4):
    temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
    ax2.append(temp)

ax_tracesy.set_xlabel("Time (sec)")
ax_tracesy.set_ylim(0,1024)
# add 
#ax.margins(x=0)

##location of sliders
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
#sfreq = Slider(axfreq, 'Frame',0.20, 18, valinit=0, dragging=True,  fc='blue')    
#sfreq.val=val


# update the slider
def update(val):
    global tracesx, tracesy

    #amp = samp.val
    index = int(sfreq.val)
    current_index[0] = index
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
    fig.canvas.draw_idle()

sfreq.on_changed(update)
#samp.on_changed(update)

# resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
# button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

# def reset(event):
#     sfreq.reset()
#     #samp.reset()
# button.on_clicked(reset)

# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.8, 0.07, 0.1], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
    
def animal_select(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    
    
animal_buttons.on_clicked(animal_select)

# ANIMAL SWITCHER
rax2 = plt.axes([0.01, 0.65, 0.07, 0.1], facecolor=axcolor)
animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax2.set_title('Switch')

def animal_switch_function(label):
    #l.set_color(label)
    global tracesx, tracesy

    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k
    print ("switching to : ", labels[switch_id])
    print ("current index inside switch:", current_index)

    # search active trace snippet
    # search forward

    def find_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
    
    end2, start2 = find_snippet(current_index, tracesx, switch_id)
            
    print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    
    end1, start1 = find_snippet(current_index, tracesx, selected_animal[0])
    
    print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
        
        
    # DATA SWITCH HAS TO BE SAFE FOR ALL DOWNSTREAM 
    # SO NEED TO CHECK THE ID THAT"S BEING SWAPPED TO MAKE SURE IT DOESN"T GET DELETED;
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()

    buff = 2
    print ("Switch ANIMAL TRACES preswitch; animal: ", switch_id,
          tracesx[start1-buff:end1+buff, switch_id],
          tracesy[start1-buff:end1+buff, switch_id])
        
        
    # replace switch id info
    tracesx[start1:end1, switch_id] = tracesx[start1:end1, selected_animal[0]].copy()
    tracesy[start1:end1, switch_id] = tracesy[start1:end1, selected_animal[0]].copy()
    
    print ("Switch ANIMAL TRACES postswitch; animal: ", switch_id, 
          tracesx[start1-buff:end1+buff, switch_id],
          tracesy[start1-buff:end1+buff, switch_id])
           
    print ("Selected ANIMAL TRACES preswitch; animal: ",selected_animal[0],
          tracesx[start1-buff:end1+buff, selected_animal[0]],
          tracesy[start1-buff:end1+buff, selected_animal[0]])

    # replace currental animal info
    tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id].copy()
    tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id].copy()
    
    print ("SelectedANIMAL TRACES postswitch; animal ",selected_animal[0],
          tracesx[start1-buff:end1+buff, selected_animal[0]],
          tracesy[start1-buff:end1+buff, selected_animal[0]])
  
    # update plots
    update(current_index[0])
    
animal_switch.on_clicked(animal_switch_function)


# search for the next continuous chunk of red
def next_segment(event):
    
    # 
    start_nan = n_frames-1
    for k in range(current_index[0], n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)
    
    start_nan+=1
    start_val = n_frames-1
    for k in range(start_nan, n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_val=k
            break

    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = start_val
    sfreq.val = current_index[0]
    fig.canvas.draw_idle()

    print ("..... to : ", current_index[0])
    update(current_index[0])
    
    # search for the next continuous chunk of red

    
def forward_function(event):
    
    # 
    current_index[0]+=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()
    
def backward_function(event):
    
    # 
    current_index[0]-=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()

        
def del_jumps_function(event):
    
    global tracesx, tracesy
    # 
    print ("DELETING JUMPS FUCNTION")
    tracesx, tracesy = delete_jumps(tracesx,tracesy)

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    
def previous_segment(event):
    
    # 
    print ("Current esgemtn: ", tracesx[current_index[0]:current_index[0]+5,
                                        selected_animal[0]])
    
    # END OF PREVIOUS NAN BLOCK
    end_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            end_nan = k
            break
    print ("searching... start_nan: ", end_nan)

    # START OF PREVIOUS NAN BLOCK
    start_nan=0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_nan=k+1
            break

    start_data =0
    for k in range(start_nan-1, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k+1 # select the very next val
            break
            
    print ("Reversing from: ", current_index[0])
    current_index[0] = start_data
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    #fig.canvas.draw_idle()

    
def save(event):
    
    np.savez('/home/cat/traces_fixed', 
             tracesx = tracesx,
             tracesy = tracesy)
    print ("SAVING TRACES")
        
def load_function(event):
    
    global tracesx, tracesy
    
    data = np.load('/home/cat/traces_fixed.npz')
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    update(current_index[0])

    print ("loaded traces")

def interpolate_function(event):
    global tracesx, tracesy
    
    dist_threshold = 50.
    time_threshold = 10
    tracesx, tracesy = interpolate_traces(tracesx, tracesy, 
                                          dist_threshold,
                                         time_threshold)
    
    update(current_index[0])
    #fig.canvas.draw_idle()
    print ("INTERPOLATING TRACES, THRESHOLD: ", threshold)

def export_movie(movie):
    clrs = ['blue','red','cyan','green']

    size_vid = np.array([1280,1024])
    dot_size = 16
    
    out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    fname_out = (out_dir+'video_labeled_exported_fixed.avi')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

    # initialize start and ends of video to be analyzed
    start = 0
    #end = start + 10*25
    end = tracesx.shape[0]
    #end = 3000
    original_vid = cv2.VideoCapture(video_name)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    for k in range(start,end,1):    
        if k %100==0:
            print (k)

        ret, frame = original_vid.read()
        if ret==False:
            break
#         if scaling:
#             frame_cropped = frame.copy()[::4,::4]
#         else:
#             frame_cropped = frame.copy()

        # Loop over animals
        for p in range(0, tracesx.shape[1], 1):
            x = tracesx[k,p]
            y = tracesy[k,p]
            # plot large bump
            if np.isnan(x) or np.isnan(y):
                continue
            x = int(x)
            y = int(y)
            
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()

def fix_nans(event):
    global tracesx, tracesy

    ids = np.arange(4)
    for id_ in ids:
        for k in range(tracesx.shape[0]):
            if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
                print (k, tracesx[k,id_], tracesy[k,id_])
                tracesx[k,id_]= tracesy[k,id_] =np.nan
                print ('')
                
    print ("DONE FIXING nans")
    
def del_singles(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-2,1):
            if k == 2310:
                print (id_, np.isnan(tracesx[k:k+3,id_]))
            if np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True, 
                                                 False,
                                                 True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan
                print (id_, k,k+3, "fixed single!")
    print (tracesx[2311-2:2311+2,2])
    print ("DONE DELEting singles")   
    
#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.01, 0.55, 0.04, 0.04])
bnext = Button(axnext, 'Next\nSegment')
bnext.on_clicked(next_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.01, 0.5, 0.04, 0.04])
bprev = Button(axprev, 'Previous\nSegment')
bprev.on_clicked(previous_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)




#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axforward = plt.axes([0.01, 0.45, 0.04, 0.04])
bforward = Button(axforward, 'Step\nForward')
bforward.on_clicked(forward_function)

# BACK BUTTON
axback = plt.axes([0.01, 0.4, 0.04, 0.04])
bback = Button(axback, 'Step\nBackward')
bback.on_clicked(backward_function)
#bprev = Button(axprev, 'Previous')

# DELETE JUMPS BUTTON
axdel_jumps = plt.axes([0.01, 0.3, 0.04, 0.04])
bdeljumps = Button(axdel_jumps, 'Fix\nJumps')
bdeljumps.on_clicked(del_jumps_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axinterpolate = plt.axes([0.01, 0.25, 0.04, 0.04])
binterpoloate = Button(axinterpolate, 'Interpolate\nTraces')
binterpoloate.on_clicked(interpolate_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axnans = plt.axes([0.01, 0.2, 0.04, 0.04])
bnans = Button(axnans, 'Fix\nNans')
bnans.on_clicked(fix_nans)
#bprev = Button(axprev, 'Previous')
# INTERPOLATE
axsingles = plt.axes([0.01, 0.15, 0.04, 0.04])
bsingles = Button(axsingles, 'Del\nSingles')
bsingles.on_clicked(del_singles)

#bprev = Button(axprev, 'Previous')

# Export movie
axmovie = plt.axes([0.01, 0.05, 0.04, 0.04])
bmovie = Button(axmovie, 'Export\nMovie')
bmovie.on_clicked(export_movie)
#bprev = Button(axprev, 'Previous')

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.06, 0.1, 0.04, 0.04])
bsave = Button(axsave, 'Save\nTraces')
bsave.on_clicked(save)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload = plt.axes([0.06, 0.05, 0.04, 0.04])
bload = Button(axload, 'Load\nTraces')
bload.on_clicked(load_function)

plt.show()

0 654 657 fixed single!
0 898 901 fixed single!
0 [False False False]
0 2680 2683 fixed single!
0 2683 2686 fixed single!
0 2856 2859 fixed single!
0 3197 3200 fixed single!
0 3205 3208 fixed single!
0 4536 4539 fixed single!
0 5110 5113 fixed single!
0 5902 5905 fixed single!
0 5905 5908 fixed single!
0 6115 6118 fixed single!
0 6759 6762 fixed single!
0 6897 6900 fixed single!
0 7313 7316 fixed single!
0 7504 7507 fixed single!
0 7668 7671 fixed single!
0 8584 8587 fixed single!
0 8629 8632 fixed single!
0 8863 8866 fixed single!
0 9078 9081 fixed single!
0 9840 9843 fixed single!
0 9870 9873 fixed single!
0 10285 10288 fixed single!
0 10360 10363 fixed single!
0 10364 10367 fixed single!
0 10785 10788 fixed single!
0 10828 10831 fixed single!
0 10862 10865 fixed single!
0 11121 11124 fixed single!
0 11124 11127 fixed single!
0 11182 11185 fixed single!
0 11671 11674 fixed single!
0 12573 12576 fixed single!
0 14151 14154 fixed single!
0 14755 14758 fixed single!
0 14770 14773 fixed 

current indx updated: [11751]
current indx updated: [11178]
current indx updated: [11030]
current indx updated: [10961]
current indx updated: [10052]
current indx updated: [9904]
current indx updated: [9696]
current indx updated: [9508]
current indx updated: [10101]
current indx updated: [10210]
current indx updated: [10220]
current indx updated: [10180]
current indx updated: [10091]
current indx updated: [10062]
current indx updated: [10042]
searching... start_nan:  10042
Fast forwarding from:  10042
..... to :  10214
current indx updated: [10214]
searching... start_nan:  10216
Fast forwarding from:  10214
..... to :  10259
current indx updated: [10259]
searching... start_nan:  10264
Fast forwarding from:  10259
..... to :  10267
current indx updated: [10267]
searching... start_nan:  10285
Fast forwarding from:  10267
..... to :  10302
current indx updated: [10302]
searching... start_nan:  10304
Fast forwarding from:  10302
..... to :  10307
current indx updated: [10307]
searching... 

switching to :  male (blue)
current index inside switch: [10717]
Switch animal id:  1 , start/end:  10702 10872
selected animal id:  3 , start/end:  10717 10778
Switch ANIMAL TRACES preswitch; animal:  1 [881.532  881.71   881.076  881.027  881.027  887.708  887.72   883.99
 883.99   883.948  890.1265 890.1265 890.1265 890.1265 899.1215 899.263
 899.263  899.263  899.263  891.052  889.245  901.749  898.746  890.347
 898.018  906.849  900.235  900.313  900.313  901.35   901.352  892.993
 892.993  893.056  893.056  893.291  905.207  895.271  895.287  903.213
 902.994  902.994  902.902  907.041  907.041  907.041  885.487  892.165
 895.2045 895.2045 895.271  895.271  895.271  895.271  898.4115 897.674
 887.991  888.701  890.459  888.574  888.481  884.954  884.889  897.248
 903.3595] [676.36   676.473  661.118  629.52   629.689  648.4655 645.4995 629.689
 628.96   628.96   644.0305 644.0305 644.0305 644.0305 651.3815 666.205
 666.205  666.205  666.205  662.635  662.475  691.929  659.787  65

switching to :  male (blue)
current index inside switch: [10717]
Switch animal id:  1 , start/end:  10702 10872
selected animal id:  3 , start/end:  10717 10778
Switch ANIMAL TRACES preswitch; animal:  1 [881.532  881.71   238.5485 238.7805 239.0745 242.005  249.6385 242.499
 242.768  257.368  257.64   245.941  252.038  258.183  258.183  258.911
 258.911  258.911  259.373  259.373  260.422  260.422  260.422  247.828
 260.422  248.492  261.192  261.192  261.417  261.641  288.378  290.319
 289.036  289.036  244.378  288.131  254.249  247.952  246.184  245.295
 245.177  242.073  242.073  242.073  243.0855 242.1705 252.205  240.498
 240.498  243.629  243.629  243.629  243.629  258.561  258.561  258.474
 257.377  257.201  256.712  238.1565 238.1565 238.1565 238.1565 897.248
 903.3595] [676.36   676.473  176.456  176.7095 177.0515 166.299  199.0685 162.728
 162.728  193.865  193.865  161.507  196.4575 188.596  188.278  188.278
 188.228  188.161  188.161  188.161  188.076  188.076  188.076  1

switching to :  female (red)
current index inside switch: [10717]
Switch animal id:  0 , start/end:  10717 10779
selected animal id:  2 , start/end:  10707 10779
Switch ANIMAL TRACES preswitch; animal:  0 [     nan      nan      nan      nan      nan      nan      nan      nan
      nan      nan      nan      nan 881.076  881.027  881.027  887.708
 887.72   883.99   883.99   883.948  890.1265 890.1265 890.1265 890.1265
 899.1215 899.263  899.263  899.263  899.263  891.052  889.245  901.749
 898.746  890.347  898.018  906.849  900.235  900.313  900.313  901.35
 901.352  892.993  892.993  893.056  893.056  893.291  905.207  895.271
 895.287  903.213  902.994  902.994  902.902  907.041  907.041  907.041
 885.487  892.165  895.2045 895.2045 895.271  895.271  895.271  895.271
 898.4115 897.674  887.991  888.701  890.459  888.574  888.481  884.954
 884.889  283.81        nan      nan] [     nan      nan      nan      nan      nan      nan      nan      nan
      nan      nan      nan      na

...

switching to :  female (red)
current index inside switch: [15910]
Switch animal id:  0 , start/end:  15910 15948
selected animal id:  3 , start/end:  15910 15915
Switch ANIMAL TRACES preswitch; animal:  0 [     nan      nan 239.0175 239.0175 239.0175 239.0175 239.0175 239.0175
 239.0175] [     nan      nan 139.5955 139.9295 139.9295 139.9295 139.9295 139.9295
 139.9295]
Switch ANIMAL TRACES postswitch; animal:  0 [     nan      nan 239.0175 239.0175 239.0175 239.0175 239.0175 239.0175
 239.0175] [     nan      nan 139.5955 139.9295 139.9295 139.9295 139.9295 139.9295
 139.9295]
Selected ANIMAL TRACES preswitch; animal:  3 [     nan      nan 239.0175 239.0175 239.0175 239.0175 239.0175      nan
      nan] [     nan      nan 139.5955 139.9295 139.9295 139.9295 139.9295      nan
      nan]
SelectedANIMAL TRACES postswitch; animal  3 [     nan      nan 239.0175 239.0175 239.0175 239.0175 239.0175      nan
      nan] [     nan      nan 139.5955 139.9295 139.9295 139.9295 139.9295      nan
 

Traceback (most recent call last):
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 215, in process
    func(*args, **kwargs)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/widgets.py", line 221, in _release
    func(event)
  File "<ipython-input-15-25443964e8fa>", line 274, in next_segment
    update(current_index[0])
  File "<ipython-input-15-25443964e8fa>", line 129, in update
    c=clrs[k])
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/__init__.py", line 1810, in inner
    return func(ax, *args, **kwargs)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/axes/_axes.py", line 1611, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/axes/_base.py", line 393, in _grab_next_args
    yield from self._plot_args(this, kwargs)
  File "/home/cat/.conda/envs/DLC-GPU/lib/

switching to :  female (red)
current index inside switch: [16479]
Switch animal id:  0 , start/end:  16480 16479
selected animal id:  3 , start/end:  16479 16503
Switch ANIMAL TRACES preswitch; animal:  0 [260.4275      nan      nan      nan      nan      nan      nan      nan
      nan      nan 843.4985 846.324  837.637  864.511  870.566  846.63
 864.595  864.7735 846.7245 871.091  871.091  871.091  854.255  846.152
 848.9955 849.111  849.076       nan] [151.374       nan      nan      nan      nan      nan      nan      nan
      nan      nan 712.226  710.2    709.541  709.9615 710.859  713.331
 713.347  713.268  713.331  715.301  715.301  714.885  714.766  714.655
 713.1125 711.343  711.8675      nan]
Switch ANIMAL TRACES postswitch; animal:  0 [260.4275      nan 267.918  260.4825 253.44   260.335  253.25   253.25
 260.335  260.3845 260.52   260.863  261.108  262.969  263.6435 259.025
 259.11   259.25   263.37   260.041  260.041  259.504  263.2795 263.2795
 263.2795 263.2795 849.076

Traceback (most recent call last):
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 215, in process
    func(*args, **kwargs)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/widgets.py", line 1032, in _clicked
    self.set_active(i)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/widgets.py", line 1064, in set_active
    func(self.labels[index].get_text())
  File "<ipython-input-15-25443964e8fa>", line 245, in animal_switch_function
    update(current_index[0])
  File "<ipython-input-15-25443964e8fa>", line 129, in update
    c=clrs[k])
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/__init__.py", line 1810, in inner
    return func(ax, *args, **kwargs)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/axes/_axes.py", line 1611, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/home/cat/.conda/envs/DLC-GP

SAVING TRACES
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500


In [14]:
print (tracesx[2311-1:2311+2,2])

[    nan 300.674     nan]


In [32]:
id_=2
for k in range(tracesx.shape[0]):
    if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
        print (k, tracesx[k,id_], tracesy[k,id_])

9318 317.532 nan
9319 317.532 nan
9320 317.532 nan
9321 317.532 nan
9322 317.532 nan
9323 317.532 nan
9324 317.532 nan
9325 317.532 nan
9326 317.532 nan
9327 317.532 nan
9328 317.532 nan
9329 317.532 nan
9789 305.184 nan


In [60]:
k = 100
id_ = 0
print (np.isnan(tracesx[k:k+3,id_]))
print (np.array([False, 
                                                 True,
                                                 False]))
print (np.all(np.isnan(tracesx[k:k+3,id_])==np.array([False, 
                                                 True,
                                                 False])))

[False False False]
[False  True False]
False
